In [1]:
import os
# 🔌 Setup and Imports
import warnings

from dotenv import load_dotenv

warnings.filterwarnings("ignore")

from datetime import datetime, timezone
from hummingbot_api_client import HummingbotAPIClient
from core.notifiers import NotificationManager, NotificationMessage

load_dotenv()

print("✅ Hummingbot Status Reporter initialized")
print("🤖 Using official hummingbot-api-client")

✅ Hummingbot Status Reporter initialized
🤖 Using official hummingbot-api-client


In [2]:
# 📊 Fetch Portfolio State and Active Bots
portfolio_state = None
active_bots_data = None
portfolio_value = 0.0
top_balances = []
active_bots = []
api_connected = False
host = os.getenv("HUMMINGBOT_API_HOST", "localhost") # Change if Hummingbot API is on a different host

print("📊 Fetching data from Hummingbot API...")

async with HummingbotAPIClient(base_url=f"http://{host}:8000") as client:
    try:
        # Get portfolio state
        try:
            portfolio_state = await client.portfolio.get_state()
            if portfolio_state and 'master_account' in portfolio_state:
                api_connected = True
                print("✅ Portfolio state retrieved")
                
                # Process portfolio data
                master_account = portfolio_state['master_account']
                
                for exchange, assets in master_account.items():
                    if isinstance(assets, list):
                        for asset_info in assets:
                            token = asset_info.get('token', 'Unknown')
                            units = float(asset_info.get('units', 0))
                            price = float(asset_info.get('price', 0))
                            value = float(asset_info.get('value', 0))
                            
                            # Only include significant balances (> $1)
                            if units > 0 and value >= 1.0:
                                top_balances.append({
                                    'asset': token,
                                    'exchange': exchange,
                                    'units': units,
                                    'price': price,
                                    'value': value
                                })
                                portfolio_value += value
                
                # Sort by value descending
                top_balances.sort(key=lambda x: x['value'], reverse=True)
                print(f"✅ Portfolio: ${portfolio_value:,.2f} in {len(top_balances)} positions")
        
        except Exception as e:
            print(f"⚠️  Portfolio state: {e}")
        
        # Get active bots status
        try:
            active_bots_data = await client.bot_orchestration.get_active_bots_status()
            if active_bots_data and active_bots_data.get('status') == 'success':
                print("✅ Active bots data retrieved")
                
                bots_data = active_bots_data.get('data', {})
                
                for bot_name, bot_info in bots_data.items():
                    if isinstance(bot_info, dict) and 'status' in bot_info:
                        bot_status = bot_info.get('status', 'Unknown')
                        performance = bot_info.get('performance', {})
                        
                        # Extract performance metrics
                        total_pnl = 0.0
                        total_volume = 0.0
                        strategies_count = len(performance)
                        
                        for strategy_name, strategy_perf in performance.items():
                            if isinstance(strategy_perf, dict) and 'performance' in strategy_perf:
                                perf_data = strategy_perf['performance']
                                total_pnl += float(perf_data.get('global_pnl_quote', 0))
                                total_volume += float(perf_data.get('volume_traded', 0))
                        
                        active_bots.append({
                            'name': bot_name,
                            'status': bot_status,
                            'strategies_count': strategies_count,
                            'total_pnl': total_pnl,
                            'total_volume': total_volume
                        })
                
                print(f"✅ Active bots: {len(active_bots)} running")
        
        except Exception as e:
            print(f"⚠️  Active bots: {e}")
            
    except Exception as e:
        print(f"❌ API Connection failed: {e}")

📊 Fetching data from Hummingbot API...
✅ Portfolio state retrieved
✅ Portfolio: $40,060.70 in 5 positions
✅ Active bots data retrieved
✅ Active bots: 1 running


In [3]:
# 📱 Generate and Send Telegram Report
notification_manager = NotificationManager()

if 'telegram' not in notification_manager.get_enabled_notifiers():
    print("❌ Telegram not configured in .env file")
else:
    print("📱 Generating Telegram report...")
    
    # Generate timestamp
    report_time = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')
    
    # Build Telegram report with HTML formatting - more concise version
    telegram_report = f"""<b>💰 Portfolio: ${portfolio_value:,.2f}</b> | 📈 Positions: {len(top_balances)}"""
    
    # Group holdings by exchange
    if top_balances:
        telegram_report += f"\n\n<b>🏆 Top Holdings:</b>"
        
        # Group balances by exchange
        exchange_balances = {}
        for balance in top_balances:
            exchange = balance['exchange']
            if exchange not in exchange_balances:
                exchange_balances[exchange] = {'total': 0, 'assets': []}
            exchange_balances[exchange]['total'] += balance['value']
            exchange_balances[exchange]['assets'].append(balance)
        
        # Sort exchanges by total value
        sorted_exchanges = sorted(exchange_balances.items(), key=lambda x: x[1]['total'], reverse=True)
        
        # Display holdings grouped by exchange
        position_counter = 1
        for exchange, data in sorted_exchanges:
            # Exchange header with total
            telegram_report += f"\n<b>{exchange.capitalize()}: ${data['total']:,.2f}</b>\n"
            
            # List assets for this exchange
            for asset in data['assets']:
                token = asset['asset']
                value = asset['value']
                telegram_report += f"<code>{position_counter:2d}. {token:>8s} | ${value:>10,.2f}</code>\n"
                position_counter += 1
    
    # Add active bots section (simplified format with volume)
    telegram_report += f"\n<b>⚙️ Active Trading Bots</b>\n"
    if active_bots:
        for bot in active_bots:
            name = bot['name']
            status = bot['status']
            pnl = bot['total_pnl']
            volume = bot['total_volume']
            
            # Status emoji indicates the status (green=running, yellow=stopped, red=error)
            status_emoji = "🟢" if status.lower() == 'running' else "🔴" if status.lower() == 'error' else "🟡"
            pnl_emoji = "📈" if pnl > 0 else "📉" if pnl < 0 else "➖"

            # Format: Status_emoji Bot_name | P&L | Volume (no status text)
            telegram_report += f"{status_emoji} {name} | {pnl_emoji} ${pnl:+.2f} | vol: ${volume:,.0f}\n\n"
    else:
        telegram_report += f"⚠️ No active bots detected\n"
    
    # Send report (no footer for cleaner look)
    try:
        telegram_notifier = notification_manager.get_notifier('telegram')
        
        notification = NotificationMessage(
            title="🤖 Hummingbot Status Report",
            message=telegram_report,
            level="info"
        )
        
        success = await telegram_notifier.send_notification(notification)
        
        if success:
            print("✅ Report sent to Telegram successfully!")
            
            # Calculate total P&L
            total_pnl = sum(bot['total_pnl'] for bot in active_bots)
            print(f"📊 Summary: {len(active_bots)} bots • ${portfolio_value:,.2f} • P&L: ${total_pnl:+.2f}")
        else:
            print("❌ Failed to send Telegram report")
            
    except Exception as e:
        print(f"❌ Telegram error: {e}")

📱 Generating Telegram report...
✅ Report sent to Telegram successfully!
📊 Summary: 1 bots • $40,060.70 • P&L: $+0.00
